嵌入（Embeddings）是将一段文本转化为向量表示的过程。可以用来做相似文本搜索的操作。
有很多嵌入模型提供商（如OpenAI、Cohere、Hugging Face等），Embedding设计了一个通用的接口；
在LangChain中，基础的“Embeddings”类暴露了两个方法：一个用于嵌入文档，另一个用于嵌入查询。前者接收多个文本作为输入，后者接收单个文本。

## Embedding使用

In [ ]:
# 会收费
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()
text = "This is a test document."
query_result = embeddings.embed_query(text)

In [60]:
doc_result = embeddings.embed_documents([text])

In [9]:
embeddings_result = embeddings.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings_result), len(embeddings_result[0])

(5, 768)

In [10]:
embedded_query = embeddings.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

[0.0951458215713501,
 9.87522435025312e-05,
 -0.01657339558005333,
 0.044848013669252396,
 0.04323705658316612]

## QA答疑

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown


In [61]:
# 加载文档
# Document loader，官方的这个链接里面的内容也很好，值得看下：
# https://lilianweng.github.io/posts/2023-06-23-agent/
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-193707.htm")
data = loader.load()


In [30]:
# 不放太多内容了，看开头大家知道是谁写的了
print(data[0].page_content[:20])



实践论：论认识和实践的关系――知和行


In [31]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)

In [21]:
print(len(all_splits))

31


In [43]:
# 遍历all_splits并检查元数据
# 过滤掉metadata为None或者包含None的文档，不然Chroma会抛异常
# 遍历all_splits并检查元数据
for doc in all_splits:
    if doc.metadata is None:
        # 如果元数据是None，设置为默认值
        doc.metadata = {"default_key": "default_value"}
    else:
        # 如果元数据不是None，但是其中的某些键或值是None，设置为默认值
        for key in list(doc.metadata.keys()):
            if doc.metadata[key] is None:
                doc.metadata[key] = "default_value"
            if key is None:
                doc.metadata["default_key"] = doc.metadata.pop(key)
print(len(all_splits))

31


In [44]:
# Store
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.redis import Redis
vectorstore = Chroma.from_documents(documents=all_splits,embedding=HuggingFaceEmbeddings())
# rds = Redis.from_documents(
#     all_splits, embeddings, redis_url="redis://localhost:6379", index_name="link"
# )

In [49]:
# 相似性搜索
question = "什么是实践论？"
docs = vectorstore.similarity_search(question)
print(len(docs))
for i, doc in enumerate(docs):
    print(f"Index {i}: {doc}")

4
Index 0: page_content='离开实践的认识是不可能的。' metadata={'language': 'default_value', 'source': 'https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-193707.htm', 'title': '实践论：论认识和实践的关系――知和行的关系（一九三七年七月）'}
Index 1: page_content='理性认识依赖于感性认识，感性认识有待于发展到理性认识，这就是辩证唯物论的认识论。哲学上的“唯理论”和“经验论”都不懂得认识的历史性或辩证性，虽然各有片面的真理（对于唯物的唯理论和经验论而言，非指唯心的唯理论和经验论），但在认识论的全体上则都是错误的。由感性到理性之辩证唯物论的认识运动，对于一个小的认识过程（例如对于一个事物或一件工作的认识）是如此，对于一个大的认识过程（例如对于一个社会或一个革命的认识）也是如此。' metadata={'language': 'default_value', 'source': 'https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-193707.htm', 'title': '实践论：论认识和实践的关系――知和行的关系（一九三七年七月）'}
Index 2: page_content='我们再来看战争。战争的领导者，如果他们是一些没有战争经验的人，对于一个具体的战争（例如我们过去十年的土地革命战争）的深刻的指导规律，在开始阶段是不了解的。他们在开始阶段只是身历了许多作战的经验，而且败仗是打得很多的。然而由于这些经验（胜仗，特别是败仗的经验），使他们能够理解贯串整个战争的内部的东西，即那个具体战争的规律性，懂得了战略和战术，因而能够有把握地去指导战争。此时，如果改换一个无经验的人去指导，又会要在吃了一些败仗之后（有了经验之后）才能理会战争的正确的规律。' metadata={'language': 'default_value', 'source': 'https://www.marxists.org/chinese/maozedong/marxist.org-chinese-mao-193

In [50]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever(),chain_type_kwargs={"verbose":True})
qa_chain({"query": question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
离开实践的认识是不可能的。

理性认识依赖于感性认识，感性认识有待于发展到理性认识，这就是辩证唯物论的认识论。哲学上的“唯理论”和“经验论”都不懂得认识的历史性或辩证性，虽然各有片面的真理（对于唯物的唯理论和经验论而言，非指唯心的唯理论和经验论），但在认识论的全体上则都是错误的。由感性到理性之辩证唯物论的认识运动，对于一个小的认识过程（例如对于一个事物或一件工作的认识）是如此，对于一个大的认识过程（例如对于一个社会或一个革命的认识）也是如此。

我们再来看战争。战争的领导者，如果他们是一些没有战争经验的人，对于一个具体的战争（例如我们过去十年的土地革命战争）的深刻的指导规律，在开始阶段是不了解的。他们在开始阶段只是身历了许多作战的经验，而且败仗是打得很多的。然而由于这些经验（胜仗，特别是败仗的经验），使他们能够理解贯串整个战争的内部的东西，即那个具体战争的规律性，懂得了战略和战术，因而能够有把握地去指导战争。此时，如果改换一个无经验的人去指导，又会要在吃了一些败仗之后（有了经验之后）才能理会战争的正确的规律。

是社会的实践。实践的观点是辩证唯物论的认识论之第一的和基本的观点[2]。
Human: 什么是实践论？

> Finished chain.

> Finished chain.


{'query': '什么是实践论？',
 'result': '实践论是辩证唯物主义的认识论观点之一，强调认识的源头和发展是通过实践来实现的。实践论认为，人们通过实际的实践活动来获取对世界的认识，而这种认识又反过来指导和改变实践活动。实践论认为，理性认识是建立在感性认识的基础上的，只有通过实践的不断发展和实践经验的积累，才能逐渐提高对世界的认识水平。实践论强调实践的历史性和辩证性，认为实践是认识的动力和基础，也是认识的检验和发展的标准。'}

In [51]:
qa_chain({"query": "怎么通过实践论提升自己？"})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
离开实践的认识是不可能的。

理性认识依赖于感性认识，感性认识有待于发展到理性认识，这就是辩证唯物论的认识论。哲学上的“唯理论”和“经验论”都不懂得认识的历史性或辩证性，虽然各有片面的真理（对于唯物的唯理论和经验论而言，非指唯心的唯理论和经验论），但在认识论的全体上则都是错误的。由感性到理性之辩证唯物论的认识运动，对于一个小的认识过程（例如对于一个事物或一件工作的认识）是如此，对于一个大的认识过程（例如对于一个社会或一个革命的认识）也是如此。

我们再来看战争。战争的领导者，如果他们是一些没有战争经验的人，对于一个具体的战争（例如我们过去十年的土地革命战争）的深刻的指导规律，在开始阶段是不了解的。他们在开始阶段只是身历了许多作战的经验，而且败仗是打得很多的。然而由于这些经验（胜仗，特别是败仗的经验），使他们能够理解贯串整个战争的内部的东西，即那个具体战争的规律性，懂得了战略和战术，因而能够有把握地去指导战争。此时，如果改换一个无经验的人去指导，又会要在吃了一些败仗之后（有了经验之后）才能理会战争的正确的规律。

充分暴露）。在这种情形之下，由于实践中发现前所未料的情况，因而部分地改变思想、理论、计划、方案的事是常有的，全部地改变的事也是有的。即是说，原定的思想、理论、计划、方案，部分地或全部地不合于实际，部分错了或全部错了的事，都是有的。许多时候须反复失败过多次，才能纠正错误的认识，才能到达于和客观过程的规律性相符合，因而才能够变主观的东西为客观的东西，即在实践中得到预想的结果。但是不管怎样，到了这种时候，人们对于在某一发展阶段内的某一客观过程的认识运动，算是完成了。
Human: 怎么通过实践论

{'query': '怎么通过实践论提升自己？',
 'result': '通过实践论，可以通过以下几个步骤来提升自己：\n\n1. 进行实践：积极参与各种实践活动，包括工作、学习、社交等。通过亲身经历和实际操作，可以更好地理解和掌握知识和技能。\n\n2. 反思和总结：在实践过程中，及时反思和总结自己的经验和教训。思考自己的行动是否达到了预期的效果，有哪些可以改进的地方，以及如何更好地应对类似的情况。\n\n3. 学习和修正：根据反思和总结的结果，学习相关的理论知识和技能，修正自己的认识和行动方式。通过学习和不断改进，提高自己的能力和水平。\n\n4. 实践和验证：将学到的知识和技能应用到实践中，验证其有效性和可行性。通过实践的结果，进一步调整和完善自己的认识和行动方式。\n\n5. 持续反馈和调整：在实践中，不断接收来自他人和环境的反馈，及时调整自己的认识和行动。通过与他人的交流和合作，不断提升自己的认知和能力。\n\n总之，通过实践论，可以通过实践、反思、学习和调整的循环过程，不断提升自己的认知和能力，实现个人的成长和发展。'}

## Chat聊天

In [52]:
from langchain.memory import ConversationBufferMemory
# ConversationBufferMemory每次会把聊天的上下文一起发给GPT
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [53]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [55]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectorstore.as_retriever()
chat = ConversationalRetrievalChain.from_llm(llm,retriever=retriever,memory=memory,verbose=True)

In [56]:
result = chat({"question": "认识的过程是什么样的?"})
result['answer']



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
离开实践的认识是不可能的。

是社会的实践。实践的观点是辩证唯物论的认识论之第一的和基本的观点[2]。

我们再来看战争。战争的领导者，如果他们是一些没有战争经验的人，对于一个具体的战争（例如我们过去十年的土地革命战争）的深刻的指导规律，在开始阶段是不了解的。他们在开始阶段只是身历了许多作战的经验，而且败仗是打得很多的。然而由于这些经验（胜仗，特别是败仗的经验），使他们能够理解贯串整个战争的内部的东西，即那个具体战争的规律性，懂得了战略和战术，因而能够有把握地去指导战争。此时，如果改换一个无经验的人去指导，又会要在吃了一些败仗之后（有了经验之后）才能理会战争的正确的规律。

为了明了基于变革现实的实践而产生的辩证唯物论的认识运动――认识的逐渐深化的运动，下面再举出几个具体的例子。
　　无产阶级对于资本主义社会的认识，在其实践的初期――破坏机器和自发斗争时期，他们还只在感性认识的阶段，只认识资本主义各个现象的片面及其外部的联系。这时，他们还是一个所谓“自在的阶级”。但是到了他们实践的第二个时期――有意识有组织的经济斗争和政治斗争的时期，由于实践，由于长期斗争的经验，经过马克思、恩格斯用科学的方法把这种种经验总结起来，产生了马克思主义的理论，用以教育无产阶级，这样就使无产阶级理解了资本主义社会的本质，理解了社会阶级的剥削关系，理解了无产阶级的历史任务，这时他们就变成了一个“自为的阶级”。
　　中国人民对于帝国主义的认识也是这样。第一阶段是表面的感性的认识阶段，表现在太平天国运动和义和团运动等笼统的排外主义的斗争上[5]。第二阶段才进到理性的认识阶段，看出了帝国主义内部和外部的各种矛盾，并看出了帝国主义

'认识的过程是一个逐渐深化的运动。在实践的初期，人们只能获得感性的认识，只能认识到事物的片面和外部联系。随着实践的不断发展和经验的积累，人们通过科学的方法总结经验，产生理论，从而逐渐理解事物的本质和内部规律。这个过程可以通过无产阶级对资本主义社会的认识和中国人民对帝国主义的认识来说明。在这个过程中，人们从一个表面的感性认识阶段逐渐进入理性的认识阶段，从而实现对事物的深入理解。'

In [57]:
print(memory.buffer)

[HumanMessage(content='认识的过程是什么样的?', additional_kwargs={}, example=False), AIMessage(content='认识的过程是一个逐渐深化的运动。在实践的初期，人们只能获得感性的认识，只能认识到事物的片面和外部联系。随着实践的不断发展和经验的积累，人们通过科学的方法总结经验，产生理论，从而逐渐理解事物的本质和内部规律。这个过程可以通过无产阶级对资本主义社会的认识和中国人民对帝国主义的认识来说明。在这个过程中，人们从一个表面的感性认识阶段逐渐进入理性的认识阶段，从而实现对事物的深入理解。', additional_kwargs={}, example=False)]


In [58]:
result = chat({"question": "怎么通过科学的方法总结经验?"})
result['answer']



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 认识的过程是什么样的?
Assistant: 认识的过程是一个逐渐深化的运动。在实践的初期，人们只能获得感性的认识，只能认识到事物的片面和外部联系。随着实践的不断发展和经验的积累，人们通过科学的方法总结经验，产生理论，从而逐渐理解事物的本质和内部规律。这个过程可以通过无产阶级对资本主义社会的认识和中国人民对帝国主义的认识来说明。在这个过程中，人们从一个表面的感性认识阶段逐渐进入理性的认识阶段，从而实现对事物的深入理解。
Follow Up Input: 怎么通过科学的方法总结经验?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
理性认识依赖于感性认识，感性认识有待于发展到理性认识，这就是辩证唯物论的认识论。哲学上的“唯理论”和“经验论”都不懂得认识的历史性或辩证性，虽然各有片面的真理（对于唯物的唯理论和经验论而言，非指唯心的唯理论和经验论），但在认识论的全体上则都是错误的。由感性到理性之辩证唯物论的认识运动，对于一个小的认识过程（例如对于一个事物或一件工作的认识）是如此，对于一个大的认识过程（例如对于一个社会或一个

'通过科学的方法总结经验，可以采取以下几种方法：\n\n1. 观察和实验：通过观察和实验来获取经验数据，收集相关的观察结果和实验数据。\n\n2. 数据分析：对收集到的数据进行统计和分析，寻找其中的规律和趋势。\n\n3. 归纳和演绎：通过归纳和演绎的方法，从具体的经验中总结出普遍的规律和原理。\n\n4. 建立假设和验证：根据总结的规律和原理，建立假设，并通过实验证实或验证假设的有效性。\n\n5. 重复和验证：通过多次重复实验，验证总结的规律和原理的可靠性和普遍性。\n\n6. 与已有理论对比：将总结的规律和原理与已有的科学理论进行对比和验证，确保其与现有知识的一致性。\n\n通过以上科学方法的应用，可以对经验进行系统的总结和归纳，从而得出科学的结论和理论。'

In [59]:
print(memory.buffer)

[HumanMessage(content='认识的过程是什么样的?', additional_kwargs={}, example=False), AIMessage(content='认识的过程是一个逐渐深化的运动。在实践的初期，人们只能获得感性的认识，只能认识到事物的片面和外部联系。随着实践的不断发展和经验的积累，人们通过科学的方法总结经验，产生理论，从而逐渐理解事物的本质和内部规律。这个过程可以通过无产阶级对资本主义社会的认识和中国人民对帝国主义的认识来说明。在这个过程中，人们从一个表面的感性认识阶段逐渐进入理性的认识阶段，从而实现对事物的深入理解。', additional_kwargs={}, example=False), HumanMessage(content='怎么通过科学的方法总结经验?', additional_kwargs={}, example=False), AIMessage(content='通过科学的方法总结经验，可以采取以下几种方法：\n\n1. 观察和实验：通过观察和实验来获取经验数据，收集相关的观察结果和实验数据。\n\n2. 数据分析：对收集到的数据进行统计和分析，寻找其中的规律和趋势。\n\n3. 归纳和演绎：通过归纳和演绎的方法，从具体的经验中总结出普遍的规律和原理。\n\n4. 建立假设和验证：根据总结的规律和原理，建立假设，并通过实验证实或验证假设的有效性。\n\n5. 重复和验证：通过多次重复实验，验证总结的规律和原理的可靠性和普遍性。\n\n6. 与已有理论对比：将总结的规律和原理与已有的科学理论进行对比和验证，确保其与现有知识的一致性。\n\n通过以上科学方法的应用，可以对经验进行系统的总结和归纳，从而得出科学的结论和理论。', additional_kwargs={}, example=False)]
